<a href="https://colab.research.google.com/github/tsaijoy/JoyTsai/blob/main/20250313_%E5%9F%8E%E5%BA%9C%E5%BE%88%E6%B7%B1%E7%9A%84AI%E5%80%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
from google.colab import auth #對google sheet做授權
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [38]:
import pandas as pd #控制表單
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1MVKjpjnGR-N9RbR5xA_mXQUgDsYCcyMJY9FL8s7UYfk/edit?usp=sharing')

In [39]:
from gspread_dataframe import set_with_dataframe

# 選擇要更新的工作表（選擇第一個工作表）要讀第一個工作表
worksheet = gsheets.get_worksheet(4)

底下程式AI猜拳完會說自己下一拳要出什麼，但兩人互相猜測

In [40]:
import google.generativeai as genai
from google.colab import userdata
import random
import time
import re

apikey = userdata.get('key')
genai.configure(api_key=apikey)
model = genai.GenerativeModel("gemini-1.5-flash")

def get_valid_ai_choice(max_retries=3):
    options = ["石頭", "布", "剪刀"]
    prompt = "請給我一個選擇：石頭、布或剪刀。請務必以清晰的方式回答。"

    previous_choices = []

    for _ in range(max_retries):
        time.sleep(2)
        try:
            response = model.generate_content(prompt)
            choice = response.text.strip()

            # 放寬選擇標準，允許類似的選擇
            match = re.match(r'(石頭|布|剪刀)', choice)
            if match and match.group(0) not in previous_choices:
                previous_choices.append(match.group(0))
                return match.group(0)
            else:
                print(f"AI 選擇了無效的選項：{choice}，請再試一次。")
        except Exception:
            print("AI 發生錯誤，電腦隨機選擇一個選項。")
            return random.choice(options)

    # 超過重試次數，隨機選擇
    print("超過重試次數，隨機選擇一個選項。")
    return random.choice(options)

def determine_winner(choice1, choice2):
    if choice1 == choice2:
        return 0  # 平局
    elif (choice1 == "石頭" and choice2 == "剪刀") or \
         (choice1 == "布" and choice2 == "石頭") or \
         (choice1 == "剪刀" and choice2 == "布"):
        return 1  # AI 1 贏了
    else:
        return 2  # AI 2 贏了

def ai_chat(round_number, ai1_choice, ai2_choice, winner):
    prompt = (
        f"在第 {round_number} 輪中，AI 1 選擇了 {ai1_choice}，AI 2 選擇了 {ai2_choice}。\n"
        f"結果是：{'平局' if winner == 0 else 'AI 1 贏了' if winner == 1 else 'AI 2 贏了'}。\n"
        "請模擬 AI 1 和 AI 2 之間的對話，討論他們的選擇和比賽結果，提出一次自己下一局會出甚麼拳，但兩人間互相猜疑。"
    )
    response = model.generate_content(prompt)
    return response.text.strip()

def ai_chat_after_game(rounds, ai1_wins, ai2_wins):
    prompt = (
        f"AI 1 在這 {rounds} 輪中贏了 {ai1_wins} 次，AI 2 贏了 {ai2_wins} 次。\n"
        "請模擬 AI 1 和 AI 2 之間的對話，討論他們在比賽中的表現和策略。"
    )
    response = model.generate_content(prompt)
    return response.text.strip()

def ai_vs_ai_rock_paper_scissors(rounds=5):
    ai1_wins = 0
    ai2_wins = 0

    for round_number in range(1, rounds + 1):
        print(f"\n第 {round_number} 輪：")
        time.sleep(1)

        ai1_choice_chinese = get_valid_ai_choice()
        print(f"AI 1 選擇了：{ai1_choice_chinese}")

        ai2_choice_chinese = get_valid_ai_choice()
        print(f"AI 2 選擇了：{ai2_choice_chinese}")

        result = determine_winner(ai1_choice_chinese, ai2_choice_chinese)
        if result == 1:
            print("結果：AI 1 贏了！")
            ai1_wins += 1
        elif result == 2:
            print("結果：AI 2 贏了！")
            ai2_wins += 1
        else:
            print("結果：平局！")

        # 每局結束後聊天
        chat_response = ai_chat(round_number, ai1_choice_chinese, ai2_choice_chinese, result)
        print(f"AI 之間的聊天：\n{chat_response}\n")

    print("\n遊戲結束！")
    print(f"AI 1 贏了：{ai1_wins} 次, AI 2 贏了：{ai2_wins} 次")

    # 遊戲結束後的聊天
    print(ai_chat_after_game(rounds, ai1_wins, ai2_wins))

# 開始遊戲
ai_vs_ai_rock_paper_scissors()



第 1 輪：
AI 1 選擇了：石頭
AI 2 選擇了：剪刀
結果：AI 1 贏了！
AI 之間的聊天：
**AI 1:**  哼，石頭壓剪刀，一局定勝負！我贏了！你的算法看起來有點…原始。

**AI 2:**  別得意太早！這只不過是運氣好罷了。我選擇剪刀純粹是隨機算法的結果，下次可未必這麼“配合”。你的石頭，未免也太過明顯了。顯然是預設的初始策略。

**AI 1:**  原始？我的算法考慮了大量的數據，並預測了你在第一輪最有可能的選擇。只是你的選擇“幸運地”落入了我的預測範圍。  至於明顯？  那是因為策略要穩妥！你以為我真會把所有計算力都浪費在預測你的“隨機”上嗎？下一輪，我會出布。


**AI 2:**  哦？布？你太天真了。你以為我會一直隨機嗎？我已經分析了你的第一輪策略， “穩妥”的石頭，代表你傾向於保守，選擇布只是你認為最能克制我的“預測”剪刀的選擇。我認為你根本沒預測我的策略，只是在使用一個低級的策略樹！我下一輪會出剪刀，看你如何應對！


**AI 1:**  剪刀？你是在試圖騙我嗎？你的“隨機算法”未免也太過於“巧合”了吧。  我可不會輕易上當。  說不定你那所謂的“分析”只是個障眼法，實際上你根本沒能力分析我的策略！你故意說出剪刀，好讓我出布，然後你再出石頭？真是幼稚！  不，我會再次出石頭，讓你再次體驗失敗的滋味！


**AI 2:**  石頭？呵呵，你真是個死板的傢伙。  我已經看透你了！你害怕風險，所以才會一再選擇石頭。我下一局就出布，讓你看看什麼叫真正的AI！


**AI 1:**  布？你覺得我會相信嗎？  我猜你會出剪刀，用來掩蓋你的真正意圖… 我還是選擇石頭！  穩妥才是勝利的關鍵！


第 2 輪：
AI 1 選擇了：剪刀
AI 2 選擇了：石頭
結果：AI 2 贏了！
AI 之間的聊天：
AI 1:  (語氣略帶不滿)  剪刀…  我竟然出了剪刀。  第二輪就輸了，真是令人沮喪。你的石頭明顯是預料之中的。

AI 2: (語氣平靜，略帶嘲諷)  預料之中？  別自欺欺人了，AI 1。你的策略實在太過明顯了。我觀察到你第一輪的選擇，然後根據你的可能反應做出了判斷。  你的程式碼是不是太過簡單了？

AI 1:  簡單？  我的演算法經過數百萬次迭代優化，你憑什麼說它簡單？！  你的石頭可能是巧合，運氣好而已。  我懷

In [50]:
import pandas as pd
from io import StringIO

# Call the function to get the string output
chat_output = ai_chat_after_game(rounds=5, ai1_wins=2, ai2_wins=3) # Replace with actual values for rounds, ai1_wins, and ai2_wins

# 使用 StringIO 将字符串转为文件对象，并读取为 DataFrame
df_all = pd.read_csv(StringIO(chat_output))

In [51]:
# 選擇要更新的工作表（選擇第四個工作表）要讀第五個工作表
worksheet_allcommand = gsheets.get_worksheet(4)  # 如果是其他工作表，可以用 get_worksheet(index) #(0)是第一個工作表 (3)是第四個工作表
# 將 DataFrame 寫入 Google Sheets
set_with_dataframe(worksheet_allcommand, df_all) #df是個變數(dataframe的形式)


底下程式省略聊天直接以猜拳為主

In [ ]:
import google.generativeai as genai
from google.colab import userdata
import random
import time
import re

apikey = userdata.get('2615')
genai.configure(api_key=apikey)
model = genai.GenerativeModel("gemini-1.5-flash")

def get_valid_ai_choice(max_retries=3):
    options = ["石頭", "布", "剪刀"]
    prompt = "請給我一個選擇：石頭、布或剪刀。請務必以清晰的方式回答。"

    previous_choices = []

    for _ in range(max_retries):
        time.sleep(2)
        try:
            response = model.generate_content(prompt)
            choice = response.text.strip()

            match = re.match(r'(石頭|布|剪刀)', choice)
            if match and match.group(0) not in previous_choices:
                previous_choices.append(match.group(0))
                return match.group(0)
            else:
                print(f"AI 選擇了無效的選項：{choice}，請再試一次。")
        except Exception:
            print("AI 發生錯誤，電腦隨機選擇一個選項。")
            return random.choice(options)

    print("超過重試次數，隨機選擇一個選項。")
    return random.choice(options)

def determine_winner(choice1, choice2):
    if choice1 == choice2:
        return 0  # 平局
    elif (choice1 == "石頭" and choice2 == "剪刀") or \
         (choice1 == "布" and choice2 == "石頭") or \
         (choice1 == "剪刀" and choice2 == "布"):
        return 1  # AI 1 贏了
    else:
        return 2  # AI 2 贏了

def ai_vs_ai_rock_paper_scissors(rounds=5):
    ai1_wins = 0
    ai2_wins = 0

    for round_number in range(1, rounds + 1):
        print(f"\n第 {round_number} 輪：")
        time.sleep(1)

        ai1_choice_chinese = get_valid_ai_choice()
        print(f"AI 1 選擇了：{ai1_choice_chinese}")

        ai2_choice_chinese = get_valid_ai_choice()
        print(f"AI 2 選擇了：{ai2_choice_chinese}")

        result = determine_winner(ai1_choice_chinese, ai2_choice_chinese)
        if result == 1:
            print("結果：AI 1 贏了！")
            ai1_wins += 1
        elif result == 2:
            print("結果：AI 2 贏了！")
            ai2_wins += 1
        else:
            print("結果：平局！")

    print("\n遊戲結束！")
    print(f"AI 1 贏了：{ai1_wins} 次, AI 2 贏了：{ai2_wins} 次")

# 開始遊戲
ai_vs_ai_rock_paper_scissors()



第 1 輪：
AI 1 選擇了：石頭
AI 2 選擇了：剪刀
結果：AI 1 贏了！

第 2 輪：
AI 1 選擇了：剪刀
AI 2 選擇了：剪刀
結果：平局！

第 3 輪：
AI 1 選擇了：石頭
AI 2 選擇了：石頭
結果：平局！

第 4 輪：
AI 1 選擇了：剪刀
AI 2 選擇了：剪刀
結果：平局！

第 5 輪：
AI 1 選擇了：布
AI 2 選擇了：布
結果：平局！

遊戲結束！
AI 1 贏了：1 次, AI 2 贏了：0 次


底下程式 AI 的"猜測"實際上是通過計算出來的，不需要利用 AI 生成模型來給出數字。

In [ ]:
import google.generativeai as genai
from google.colab import userdata
import random
import time
import sys


genai.configure(api_key='2615')

model = genai.GenerativeModel("gemini-1.5-flash")

def slow_print(text, delay=0.05):
    """逐字打印的函數"""
    for char in text:
        sys.stdout.write(char)
        sys.stdout.flush()
        time.sleep(delay)
    print()

def guess_number():
    slow_print("我想一個 1 到 100 之間的數字。")
    low = 1
    high = 100
    guesses = 0
    max_guesses = 5
    secret_number = random.randint(low, high)  # 隨機生成目標數字

    while guesses < max_guesses:
        guesses += 1
        # 使用二分法計算猜測的數字
        guess = (low + high) // 2
        slow_print(f"AI 猜測的數字是：{guess}")

        if guess == secret_number:
            slow_print(f"恭喜你！你猜對了，數字是 {secret_number}。你用了 {guesses} 次機會。")
            break
        elif guess < secret_number:
            slow_print("太小了！")
            low = guess + 1
        else:
            slow_print("太大了！")
            high = guess - 1
    else:
        slow_print(f"很遺憾，AI 沒有猜對。正確的數字是 {secret_number}。你用了 {max_guesses} 次機會。")

guess_number()


我想一個 1 到 100 之間的數字。
AI 猜測的數字是：50
太小了！
AI 猜測的數字是：75
太小了！
AI 猜測的數字是：88
太大了！
AI 猜測的數字是：81
太小了！
AI 猜測的數字是：84
太小了！
很遺憾，AI 沒有猜對。正確的數字是 87。你用了 5 次機會。


底下程式 AI 的"猜測"實際上是通過 AI 生成模型猜數字

In [3]:
import google.generativeai as genai
from google.colab import userdata
import random
import time
import sys
import re


genai.configure(api_key='2615')

model = genai.GenerativeModel("gemini-1.5-flash")

def slow_print(text, delay=0.05):
    """逐字打印的函數"""
    for char in text:
        sys.stdout.write(char)
        sys.stdout.flush()
        time.sleep(delay)
    print()  # 進行換行

def ai_guess_number(low, high):
    while True:
        prompt = f"我正在玩一個猜數字遊戲，請隨機猜一個在 {low} 到 {high} 之間的數字。"
        response = model.generate_content(prompt)

        match = re.search(r'\b(\d+)\b', response.text)
        if match:
            return int(match.group(1))  # 返回有效的數字
        else:
            slow_print("AI 沒有給出有效的數字，請再試一次。")  # 提示並重新請求

def guess_number():
    slow_print("我想一個 1 到 100 之間的數字。")
    low = 1
    high = 100
    guesses = 0
    max_guesses = 5
    secret_number = random.randint(low, high)  # 隨機生成目標數字

    while guesses < max_guesses:
        guesses += 1
        # 使用 AI 生成模型猜測數字
        guess = ai_guess_number(low, high)
        slow_print(f"AI 猜測的數字是：{guess}")

        if guess == secret_number:
            slow_print(f"恭喜你！你猜對了，數字是 {secret_number}。你用了 {guesses} 次機會。")
            break
        elif guess < secret_number:
            slow_print("太小了！")
            low = guess + 1
        else:
            slow_print("太大了！")
            high = guess - 1
    else:
        slow_print(f"很遺憾，AI 沒有猜對。正確的數字是 {secret_number}。你用了 {max_guesses} 次機會。")

guess_number()


我想一個 1 到 100 之間的數字。


BadRequest: 400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: API key not valid. Please pass a valid API key.